In [2]:
from collections import Counter

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from pypots.utils.random import set_random_seed
from pypots.optim import Adam
from pypots.classification import Raindrop, BRITS, GRUD
from pypots.nn.functional import calc_binary_classification_metrics

# Utility functions

## 📍 Station Information

| Field        | Description                                      |
|--------------|--------------------------------------------------|
| `STN`        | Station number                                   |
| `LON(east)`  | Longitude (degrees east)                         |
| `LAT(north)` | Latitude (degrees north)                         |
| `ALT(m)`     | Altitude (in meters)                             |
| `NAME`       | Station name                                     |

## 📅 Time Information

| Field       | Description |
|-------------|-------------|
| `YYYYMMDD`  | Date (YYYY = year; MM = month; DD = day) |
| `HH`        | Hour (HH = hour; UT time. E.g., 12 UT = 13 MET, 14 MEZT. Hour block 05 = 04:00–05:00 UT) |

## 🌬️ Wind Data

| Field | Description |
|-------|-------------|
| `DD`  | Wind direction (in degrees), averaged over the last 10 minutes of the previous hour<br>(360 = north, 90 = east, 180 = south, 270 = west, 0 = calm, 990 = variable) |
| `FH`  | Hourly mean wind speed (in 0.1 m/s) |
| `FF`  | Mean wind speed (in 0.1 m/s) during last 10 minutes of the previous hour |
| `FX`  | Maximum wind gust (in 0.1 m/s) during the hour block |

## 🌡️ Temperature & Humidity

| Field   | Description |
|---------|-------------|
| `T`     | Temperature (in 0.1 °C) at 1.50 m height at observation time |
| `T10N`  | Minimum temperature (in 0.1 °C) at 10 cm height in the past 6 hours |
| `TD`    | Dew point temperature (in 0.1 °C) at 1.50 m at observation time |
| `U`     | Relative humidity (%) at 1.50 m height |

## 🌞 Radiation & Sunshine

| Field | Description |
|-------|-------------|
| `SQ`  | Sunshine duration (in 0.1 hours) per hour block, calculated from global radiation<br>(-1 = less than 0.05 h) |
| `Q`   | Global radiation (in J/cm²) per hour block |

## 🌧️ Precipitation

| Field | Description |
|-------|-------------|
| `DR`  | Duration of precipitation (in 0.1 hours) per hour block |
| `RH`  | Hourly precipitation amount (in 0.1 mm)<br>(-1 = less than 0.05 mm) |

## 🌬️ Atmospheric Pressure

| Field | Description |
|-------|-------------|
| `P`   | Air pressure (in 0.1 hPa), reduced to sea level, at observation time |

## 👁️ Visibility & Clouds

| Field | Description |
|-------|-------------|
| `VV`  | Horizontal visibility at observation time<br>(0 = <100 m, 1 = 100–200 m, ..., 89 = >70 km) |
| `N`   | Cloud cover (in octants, 0–8); 9 = sky invisible |

## 🌦️ Weather Conditions

| Field | Description |
|-------|-------------|
| `WW`  | Present weather code (00–99); observed visually or automatically |
| `IX`  | Weather code observation type:<br>1 = manned/visual; 2/3 = manned/omitted;<br>4 = automatic/visual; 5/6 = automatic/omitted; 7 = automatic sensor-based |

## 🌫️ Weather Phenomena Indicators

| Field | Description |
|-------|-------------|
| `M`   | Fog: 0 = none; 1 = occurred during the previous hour and/or observation |
| `R`   | Rain: 0 = none; 1 = occurred during the previous hour and/or observation |
| `S`   | Snow: 0 = none; 1 = occurred during the previous hour and/or observation |
| `O`   | Thunder: 0 = none; 1 = occurred during the previous hour and/or observation |
| `Y`   | Ice formation: 0 = none; 1 = occurred during the previous hour and/or observation |


In [15]:
def prepare_df(path: str) -> pd.DataFrame:
    try:
        header_line_index = -1
        column_names = []
        data_lines_start_index = -1

        # Find the header and its index more efficiently
        with open(path, 'r') as f:
            for i, line in enumerate(f):
                if line.strip().startswith('# STN,YYYYMMDD,'):
                    header_line_index = i
                    column_names = [col.strip() for col in line.strip().lstrip('#').split(',')]
                    data_lines_start_index = header_line_index + 1
                    break
        
        if header_line_index == -1:
            raise ValueError("Header line not found.")

        # Use pandas.read_csv directly with skiprows and comment character
        # This avoids reading the whole file into a list first for data lines
        # and then joining them back.
        df = pd.read_csv(
            path,
            names=column_names,
            skiprows=data_lines_start_index,
            comment='#',  # Lines starting with '#' will be ignored as comments
            skipinitialspace=True,
            na_values=['       ', '     '] # Add other common missing value representations if needed
        )

        if df.empty:
            raise ValueError("No data found after the header or all data was commented out.")

        # Convert 'HH' to string and zfill, then create 'Timestamp'
        # It's crucial to handle potential NaN values in 'YYYYMMDD' or 'HH'
        # if they are not guaranteed to be present or valid in all rows.
        df['HH'] = df['HH'].astype(int) - 1
        df['HH'] = df['HH'].astype(str).str.zfill(2)
        df['Timestamp'] = pd.to_datetime(df['YYYYMMDD'].astype(str) + df['HH'].astype(str), format="%Y%m%d%H", errors='coerce')
        
        df.set_index('Timestamp', inplace=True)
        
        # Columns to drop
        # cols_to_drop = ['YYYYMMDD', 'HH']
        # df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)

        # Convert remaining columns to numeric, efficiently
        # Identify numeric columns once and convert
        # Exclude already processed or known non-numeric columns if necessary
        for col in df.columns:
            # This check is slightly redundant if YYYYMMDD and HH are already dropped,
            # but good for safety if they weren't or if other non-numeric columns exist.
            if df[col].dtype == 'object': # Only attempt conversion if the column is of object type
                try:
                    df[col] = pd.to_numeric(df[col], downcast='signed')
                except ValueError:
                    # Handle or log cases where a column expected to be numeric isn't
                    # For now, we'll coerce, which turns unparseable into NaT/NaN
                    df[col] = pd.to_numeric(df[col], errors='coerce', downcast='signed')
        
        return df

    except FileNotFoundError:
        print(f"Error: The file '{path}' was not found.")
        raise
    except ValueError as ve:
        print(f"ValueError: {ve}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise

df = prepare_df("./datasets/knmi_station_data/215.txt")
df.head()

,STN,YYYYMMDD,HH,DD,FH,FF,FX,T,T10N,TD,...,VV,N,U,WW,IX,M,R,S,O,Y
Timestamp,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,215,20150101,0,210.0,50.0,50.0,70.0,27,NaN,8,...,42.0,0.0,87,10.0,7,0.0,0.0,0.0,0.0,0.0
2015-01-01 01:00:00,215,20150101,1,220.0,50.0,50.0,70.0,26,NaN,4,...,57.0,0.0,85,10.0,7,0.0,0.0,0.0,0.0,0.0
2015-01-01 02:00:00,215,20150101,2,200.0,50.0,40.0,80.0,23,NaN,2,...,60.0,0.0,86,NaN,5,0.0,0.0,0.0,0.0,0.0
2015-01-01 03:00:00,215,20150101,3,210.0,40.0,40.0,70.0,21,NaN,1,...,60.0,0.0,87,NaN,5,0.0,0.0,0.0,0.0,0.0
2015-01-01 04:00:00,215,20150101,4,190.0,50.0,50.0,80.0,19,NaN,2,...,60.0,1.0,88,NaN,5,0.0,0.0,0.0,0.0,0.0


In [16]:
# Horizontal visibility at observation time
# (0 = <100 m, 1 = 100–200 m, ..., 89 = >70 km)
df['VV'].value_counts()

VV
75.0    4402
70.0    4249
65.0    2454
80.0    2269
63.0    1196
        ... 
15.0      51
8.0       49
14.0      48
9.0       42
10.0      40
Name: count, Length: 79, dtype: int64

In [25]:
df[df['VV'] == 12].head()

,STN,YYYYMMDD,HH,DD,FH,FF,FX,T,T10N,TD,...,VV,N,U,WW,IX,M,R,S,O,Y
Timestamp,,,,,,,,,,,,,,,,,,,,,
2015-01-23 04:00:00,215,20150123,4,0.0,0.0,0.0,10.0,-53,NaN,-60,...,12.0,0.0,94,10.0,7,0.0,0.0,0.0,0.0,0.0
2015-01-23 06:00:00,215,20150123,6,0.0,10.0,0.0,10.0,-52,NaN,-57,...,12.0,0.0,96,10.0,7,0.0,0.0,0.0,0.0,0.0
2015-02-17 01:00:00,215,20150217,1,240.0,20.0,20.0,50.0,35,NaN,32,...,12.0,8.0,98,20.0,7,1.0,0.0,0.0,0.0,0.0
2015-02-17 05:00:00,215,20150217,5,210.0,10.0,10.0,20.0,42,26.0,37,...,12.0,8.0,97,10.0,7,0.0,0.0,0.0,0.0,0.0
2015-04-07 07:00:00,215,20150407,7,300.0,30.0,30.0,40.0,56,NaN,53,...,12.0,8.0,98,20.0,7,1.0,0.0,0.0,0.0,0.0


In [31]:
for col in df.columns:
    print(f"{col}: {len(df[col].value_counts())}")

STN: 1
DD: 38
FH: 21
FF: 22
FX: 33
T: 353
T10N: 295
TD: 307
SQ: 12
Q: 323
DR: 11
RH: 67
P: 711
VV: 79
N: 10
U: 76
WW: 41
IX: 3
M: 2
R: 2
S: 2
O: 2
Y: 2


In [32]:
df["DD"].value_counts()

DD
200.0    1768
230.0    1692
240.0    1682
210.0    1541
250.0    1530
220.0    1324
260.0    1264
190.0    1253
270.0    1110
180.0    1101
60.0     1092
70.0     1090
340.0    1056
280.0    1055
330.0    1047
320.0    1040
80.0     1036
350.0    1030
170.0     953
110.0     943
290.0     910
50.0      898
310.0     891
130.0     785
360.0     775
140.0     758
100.0     754
300.0     753
120.0     747
10.0      739
40.0      721
30.0      714
0.0       625
90.0      598
20.0      591
160.0     580
150.0     532
990.0     241
Name: count, dtype: int64

In [15]:
def get_na_stats(df, col):
    na = df[col].isna().sum()
    with_value = df[col].fillna(0).astype(bool).sum()
    print(f"{col}: {na}, {with_value}, {len(df)}")

In [22]:
get_na_stats(df, 'M')
get_na_stats(df, 'T10N')

M: 203, 1839, 37248
T10N: 31040, 6172, 37248


In [57]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder # Ensure this is imported

# Helper function for core processing and sequence creation
def _process_and_create_sequences_internal(
        df_segment: pd.DataFrame,
        target_column_name: str,
        numerical_features_cols: list[str],
        categorical_features_cols: dict[str, OneHotEncoder],
        prev_time_steps: int,
) -> dict[str, np.ndarray]:

    # This df_segment is assumed to have target_column_name NaNs already handled if desired.
    
    processed_feature_dfs_list = []

    # 1. Process Numerical Features
    if numerical_features_cols:
        valid_numerical_cols = [col for col in numerical_features_cols if col in df_segment.columns]
        if valid_numerical_cols: # Only proceed if there are valid numerical columns to select
            numerical_df = df_segment[valid_numerical_cols].copy()
            processed_feature_dfs_list.append(numerical_df)
        elif not valid_numerical_cols and numerical_features_cols: # Specified but none found
            print(f"Warning (internal): None of the specified numerical features found in the current data segment. Numerical features count: {len(numerical_features_cols)}")


    # 2. Process Categorical Features
    for col_name, encoder in categorical_features_cols.items():
        if col_name not in df_segment.columns:
            # This happens if df_segment is empty or the column was dropped.
            # The impact on num_actual_features will be handled later.
            continue

        column_to_encode = df_segment[[col_name]]
        encoded_data_sparse = np.array([]) # Default empty
        
        if column_to_encode.empty: # Encoder might not handle empty DataFrame input well for transform if it expects rows
             try: # Get feature names to create an empty DataFrame with correct OHE columns
                ohe_feature_names = encoder.get_feature_names_out([col_name])
             except AttributeError:
                ohe_feature_names = encoder.get_feature_names([col_name])
             except Exception:
                ohe_feature_names = [f"{col_name}_cat{i}" for i in range(len(encoder.categories_[0]))] if hasattr(encoder, 'categories_') else [f"{col_name}_unknown_cat"]
             
             # Create an empty DataFrame (0 rows) with the OHE column names
             ohe_df = pd.DataFrame(columns=ohe_feature_names, index=df_segment.index, dtype=float)
             processed_feature_dfs_list.append(ohe_df)
             continue # Go to next categorical column

        # If not empty, proceed with transform
        encoded_data_sparse = encoder.transform(column_to_encode)
        
        try:
            ohe_feature_names = encoder.get_feature_names_out([col_name])
        except AttributeError:
            try:
                ohe_feature_names = encoder.get_feature_names([col_name])
            except TypeError:
                ohe_feature_names = [f"{col_name}_{category}" for category in encoder.categories_[0]]
            except Exception as e_fn: # Broad exception for get_feature_names issues
                print(f"Warning (internal): Could not reliably get OHE feature names for '{col_name}'. Using generic names. Error: {e_fn}")
                num_output_features = encoded_data_sparse.shape[1]
                ohe_feature_names = [f"{col_name}_ohe_{i}" for i in range(num_output_features)]

        if hasattr(encoded_data_sparse, "toarray"):
            encoded_data_dense = encoded_data_sparse.toarray()
        else:
            encoded_data_dense = encoded_data_sparse
        
        if encoded_data_dense.shape[0] > 0 : # Ensure data was actually produced
            ohe_df = pd.DataFrame(encoded_data_dense, columns=ohe_feature_names, index=df_segment.index)
            processed_feature_dfs_list.append(ohe_df)
        elif df_segment.shape[0] > 0 : # Input segment had rows, but OHE produced no rows (should not happen with sklearn)
             print(f"Warning (internal): OHE for '{col_name}' produced 0 rows from a non-empty segment. Check encoder.")


    # 3. Combine all processed feature DataFrames
    num_actual_features = 0
    if not processed_feature_dfs_list:
        final_features_df = pd.DataFrame(index=df_segment.index) # 0 columns
        if numerical_features_cols or categorical_features_cols:
             print("Warning (internal): No features were processed into final_features_df despite specification. X will have 0 features for this segment.")
    else:
        final_features_df = pd.concat(processed_feature_dfs_list, axis=1)
    num_actual_features = final_features_df.shape[1]


    # --- Sequence Creation ---
    X_list = []
    y_list = []
    num_rows_in_features = len(final_features_df)

    if prev_time_steps >= num_rows_in_features :
        return {'X': np.array([]).reshape(0, prev_time_steps, num_actual_features), 'y': np.array([])}

    y_series_segment = df_segment[target_column_name]

    for i in range(prev_time_steps, num_rows_in_features):
        feature_sequence = final_features_df.iloc[i - prev_time_steps : i].values
        X_list.append(feature_sequence)
        target_value = y_series_segment.iloc[i]
        y_list.append(target_value)

    if not X_list:
        return {'X': np.array([]).reshape(0, prev_time_steps, num_actual_features), 'y': np.array([])}
        
    X_np = np.array(X_list)
    y_np = np.array(y_list)
    
    return {'X': X_np, 'y': y_np}

# Helper for balancing data (undersampling)
def _balance_data_helper(X_in: np.ndarray, y_in: np.ndarray, random_seed: int | None = None) -> tuple[np.ndarray, np.ndarray]:
    if X_in.shape[0] == 0: # No data to balance
        return X_in, y_in

    unique_classes, counts = np.unique(y_in, return_counts=True)
    
    if len(unique_classes) <= 1: # Already balanced or only one class
        return X_in, y_in

    min_count = np.min(counts)
    
    balanced_indices_list = []
    rng = np.random.RandomState(random_seed) # For reproducible undersampling if seed is provided

    for cls_val in unique_classes:
        cls_indices = np.where(y_in == cls_val)[0]
        if len(cls_indices) > min_count:
            chosen_indices = rng.choice(cls_indices, size=min_count, replace=False)
        else:
            chosen_indices = cls_indices # Take all if it's already the min count or less
        balanced_indices_list.extend(chosen_indices)
    
    # Shuffle the combined indices from different classes
    shuffled_balanced_indices = rng.permutation(balanced_indices_list)
    
    return X_in[shuffled_balanced_indices], y_in[shuffled_balanced_indices]


# Main Function
def create_sequences_for_classification(
        df: pd.DataFrame,
        target_column_name: str,
        numerical_features_cols: list[str],
        categorical_features_cols: dict[str, OneHotEncoder], # {col_name: fitted_encoder}
        prev_time_steps: int = 8,
        split_date: str | pd.Timestamp | None = None,
        balance_data: bool = False,
        balance_random_seed: int | None = None, # Seed for balancing reproducibility
) -> dict[str, np.ndarray] | tuple[dict[str, np.ndarray], dict[str, np.ndarray]]:

    if target_column_name not in df.columns: # Check on original df
        raise ValueError(f"Target column '{target_column_name}' not found in DataFrame.")
    
    df_features = df.copy() # Use the name from your stub
    # --- User requested modification: remove rows where target is NaN ---
    df_features = df_features[~df_features[target_column_name].isna()]
    # --- End of modification ---

    if df_features.empty:
        print(f"DataFrame is empty after removing NaNs in target column '{target_column_name}'.")
        # The _process_and_create_sequences_internal can handle an empty df_features
        # and return correctly shaped empty arrays.
        empty_sequences = _process_and_create_sequences_internal(
            df_features, target_column_name, numerical_features_cols,
            categorical_features_cols, prev_time_steps
        )
        if split_date:
            return empty_sequences, empty_sequences.copy() # Return two empty dicts
        else:
            return empty_sequences


    # Basic Validations for other parameters
    if not isinstance(prev_time_steps, int) or prev_time_steps <= 0:
        raise ValueError("'prev_time_steps' must be a positive integer.")
    for col in numerical_features_cols:
        if col not in df_features.columns: # Check on df_features as it's the one being processed
            print(f"Warning: Numerical feature column '{col}' not found in (cleaned) DataFrame. It will be ignored if missing.")
    for col in categorical_features_cols.keys():
        if col not in df_features.columns:
            print(f"Warning: Categorical feature column '{col}' not found in (cleaned) DataFrame. It will be ignored if missing.")
    
    if split_date:
        if not isinstance(df_features.index, pd.DatetimeIndex):
            raise ValueError("DataFrame index must be a DatetimeIndex for date-based splitting.")
        
        split_date_ts = pd.to_datetime(split_date) # Converts str or pd.Timestamp

        train_df = df_features[df_features.index <= split_date_ts].copy() # Use .copy() to avoid SettingWithCopyWarning later
        test_df = df_features[df_features.index > split_date_ts].copy()

        print(f"Original df_features shape: {df_features.shape}")
        print(f"Train data shape (after target NaN drop, before sequence creation): {train_df.shape}")
        print(f"Test data shape (after target NaN drop, before sequence creation): {test_df.shape}")

        if train_df.empty:
            print("Warning: Train DataFrame is empty after split.")
        if test_df.empty:
            print("Warning: Test DataFrame is empty after split.")

        train_sequences = _process_and_create_sequences_internal(
            train_df, target_column_name, numerical_features_cols,
            categorical_features_cols, prev_time_steps
        )
        test_sequences = _process_and_create_sequences_internal(
            test_df, target_column_name, numerical_features_cols,
            categorical_features_cols, prev_time_steps
        )

        if balance_data and train_sequences['X'].shape[0] > 0:
            print(f"Balancing training data... Original counts: {dict(zip(*np.unique(train_sequences['y'], return_counts=True)))}")
            train_sequences['X'], train_sequences['y'] = _balance_data_helper(
                train_sequences['X'], train_sequences['y'], random_seed=balance_random_seed
            )
            print(f"Balanced training data shapes: X={train_sequences['X'].shape}, y={train_sequences['y'].shape}. New counts: {dict(zip(*np.unique(train_sequences['y'], return_counts=True)))}")
        if balance_data and test_sequences['X'].shape[0] > 0:
            test_sequences['X'], test_sequences['y'] = _balance_data_helper(
                test_sequences['X'], test_sequences['y'], random_seed=balance_random_seed
            )
        return train_sequences, test_sequences
    else:
        # No split, process the whole df_features
        all_sequences = _process_and_create_sequences_internal(
            df_features, target_column_name, numerical_features_cols,
            categorical_features_cols, prev_time_steps
        )
        
        if balance_data and all_sequences['X'].shape[0] > 0:
            print(f"Balancing all data... Original counts: {dict(zip(*np.unique(all_sequences['y'], return_counts=True)))}")
            all_sequences['X'], all_sequences['y'] = _balance_data_helper(
                all_sequences['X'], all_sequences['y'], random_seed=balance_random_seed
            )
            print(f"Balanced data shapes: X={all_sequences['X'].shape}, y={all_sequences['y'].shape}. New counts: {dict(zip(*np.unique(all_sequences['y'], return_counts=True)))}")

        return all_sequences

In [58]:
SEQUENCE_LENGTH = 12
STEP_SIZE = 1
TARGET_COLUMN = 'M'
NUMERICAL_COLS = [
    "FH", "FF", "FX", "T", "T10N", "TD", "SQ", "Q", "DR", "RH", "P", "U", 
    # "DD"
]
CATEGORICAL_COLS = {
    # "WW", "IX", "VV"
}

In [59]:
train_data = create_sequences_for_classification(
    df=df, 
    target_column_name=TARGET_COLUMN, 
    numerical_features_cols=NUMERICAL_COLS, 
    categorical_features_cols=CATEGORICAL_COLS, 
    prev_time_steps=SEQUENCE_LENGTH,
    balance_data=True,
    balance_random_seed=42
)

Balancing all data... Original counts: {np.float64(0.0): np.int64(35194), np.float64(1.0): np.int64(1839)}
Balanced data shapes: X=(3678, 12, 12), y=(3678,). New counts: {np.float64(0.0): np.int64(1839), np.float64(1.0): np.int64(1839)}


In [60]:
train_data, test_data = create_sequences_for_classification(
    df=df, 
    target_column_name=TARGET_COLUMN, 
    numerical_features_cols=NUMERICAL_COLS, 
    categorical_features_cols=CATEGORICAL_COLS, 
    prev_time_steps=SEQUENCE_LENGTH,
    balance_data=True,
    balance_random_seed=42,
    split_date=pd.Timestamp(year=2022, month=1, day=1)
)

Original df_features shape: (37045, 23)
Train data shape (after target NaN drop, before sequence creation): (22581, 23)
Test data shape (after target NaN drop, before sequence creation): (12920, 23)
Balancing training data... Original counts: {np.float64(0.0): np.int64(21531), np.float64(1.0): np.int64(1038)}
Balanced training data shapes: X=(2076, 12, 12), y=(2076,). New counts: {np.float64(0.0): np.int64(1038), np.float64(1.0): np.int64(1038)}


In [61]:
if train_data and train_data["X"].size > 0:
    print(f"Final Training X shape: {train_data['X'].shape}")
    print(f"Final Training y shape: {train_data['y'].shape}")
    y_int = train_data['y']
    print(f"Training y distribution: {np.unique(y_int, equal_nan=True, return_counts=True)}")
else:
    print("Training data is empty or could not be generated.")

if test_data and test_data["X"].size > 0:
    print(f"Final Test X shape: {test_data['X'].shape}")
    print(f"Final Test y shape: {test_data['y'].shape}")
    y_int = test_data['y']
    print(f"Test y distribution: {np.unique(y_int, equal_nan=True, return_counts=True)}")
else:
    print("Test data is empty or could not be generated.")

Final Training X shape: (2076, 12, 12)
Final Training y shape: (2076,)
Training y distribution: (array([0., 1.]), array([1038, 1038]))
Final Test X shape: (1330, 12, 12)
Final Test y shape: (1330,)
Test y distribution: (array([0., 1.]), array([665, 665]))


# Model training

## Raindrop

In [68]:
raindrop = Raindrop(
    n_steps=train_data['X'].shape[1],
    n_features=train_data['X'].shape[2],
    n_classes=2,
    n_layers=2,
    d_model=train_data['X'].shape[2] * 4,
    d_ffn=256,
    n_heads=2,
    dropout=0.3,
    batch_size=32,
    epochs=20,
    patience=3,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/raindrop',
    model_saving_strategy='best'
)

2025-05-21 21:39:25 [INFO]: No given device, using default device: cuda
2025-05-21 21:39:25 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/raindrop/20250521_T213925
2025-05-21 21:39:25 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/raindrop/20250521_T213925/tensorboard
2025-05-21 21:39:25 [INFO]: Using customized CrossEntropy as the training loss function.
2025-05-21 21:39:25 [INFO]: Using customized CrossEntropy as the validation metric function.
2025-05-21 21:39:25 [INFO]: Raindrop initialized with the given hyperparameters, the number of trainable parameters: 173,636


In [69]:
raindrop.fit(train_set=train_data, val_set=test_data)
results = raindrop.predict(test_data)
prediction = results['classification']
metrics = calc_binary_classification_metrics(prediction, test_data['y'])
print("Testing classification metrics: \n"
    f'ROC_AUC: {metrics["roc_auc"]}, \n'
    f'PR_AUC: {metrics["pr_auc"]},\n'
    f'F1: {metrics["f1"]},\n'
    f'Precision: {metrics["precision"]},\n'
    f'Recall: {metrics["recall"]},\n'
    f'Accuracy: {metrics["accuracy"]}'
)

2025-05-21 21:39:38 [INFO]: Epoch 001 - training loss (CrossEntropy): 0.6976, validation CrossEntropy: 0.6959
2025-05-21 21:39:48 [INFO]: Epoch 002 - training loss (CrossEntropy): 0.6946, validation CrossEntropy: 0.6930
2025-05-21 21:39:57 [INFO]: Epoch 003 - training loss (CrossEntropy): 0.6931, validation CrossEntropy: 0.6670
2025-05-21 21:40:08 [INFO]: Epoch 004 - training loss (CrossEntropy): 0.6905, validation CrossEntropy: 0.6892
2025-05-21 21:40:18 [INFO]: Epoch 005 - training loss (CrossEntropy): 0.6448, validation CrossEntropy: 0.5064
2025-05-21 21:40:29 [INFO]: Epoch 006 - training loss (CrossEntropy): 0.4857, validation CrossEntropy: 0.3308
2025-05-21 21:40:40 [INFO]: Epoch 007 - training loss (CrossEntropy): 0.4191, validation CrossEntropy: 0.3190
2025-05-21 21:40:51 [INFO]: Epoch 008 - training loss (CrossEntropy): 0.4201, validation CrossEntropy: 0.3058
2025-05-21 21:41:01 [INFO]: Epoch 009 - training loss (CrossEntropy): 0.3943, validation CrossEntropy: 0.2951
2025-05-21

Testing classification metrics: 
ROC_AUC: 0.8849624060150375, 
PR_AUC: 0.9074526868351983,
F1: 0.8927820602662929,
Precision: 0.8359580052493438,
Recall: 0.9578947368421052,
Accuracy: 0.8849624060150376


## BRITS

In [70]:
brits = BRITS(
    n_steps=train_data['X'].shape[1],
    n_features=train_data['X'].shape[2],
    n_classes=2,
    rnn_hidden_size=256,
    batch_size=32,
    epochs=20,
    patience=3,
    optimizer=Adam(lr=1e-3),
    num_workers=0,
    device=None,
    saving_path='./runs/classify/WEATHER-KNMI/brits',
    model_saving_strategy='best'
)

2025-05-21 21:42:12 [INFO]: No given device, using default device: cuda
2025-05-21 21:42:12 [INFO]: Model files will be saved to ./runs/classify/WEATHER-KNMI/brits/20250521_T214212
2025-05-21 21:42:12 [INFO]: Tensorboard file will be saved to ./runs/classify/WEATHER-KNMI/brits/20250521_T214212/tensorboard
2025-05-21 21:42:12 [INFO]: Using customized CrossEntropy as the training loss function.
2025-05-21 21:42:12 [INFO]: Using customized CrossEntropy as the validation metric function.
2025-05-21 21:42:12 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 592,612


In [71]:
brits.fit(train_set=train_data, val_set=test_data)
results = brits.predict(test_data)
prediction = results['classification']
metrics = calc_binary_classification_metrics(prediction, test_data['y'])
print("Testing classification metrics: \n"
    f'ROC_AUC: {metrics["roc_auc"]}, \n'
    f'PR_AUC: {metrics["pr_auc"]},\n'
    f'F1: {metrics["f1"]},\n'
    f'Precision: {metrics["precision"]},\n'
    f'Recall: {metrics["recall"]},\n'
    f'Accuracy: {metrics["accuracy"]}'
)

2025-05-21 21:42:32 [INFO]: Epoch 001 - training loss (CrossEntropy): 2901.0337, validation CrossEntropy: 0.6801
2025-05-21 21:42:51 [INFO]: Epoch 002 - training loss (CrossEntropy): 2373.2363, validation CrossEntropy: 0.6686
2025-05-21 21:43:06 [INFO]: Epoch 003 - training loss (CrossEntropy): 2074.8020, validation CrossEntropy: 0.6677
2025-05-21 21:43:21 [INFO]: Epoch 004 - training loss (CrossEntropy): 1931.4504, validation CrossEntropy: 0.6531
2025-05-21 21:43:37 [INFO]: Epoch 005 - training loss (CrossEntropy): 1874.9596, validation CrossEntropy: 0.6409
2025-05-21 21:43:58 [INFO]: Epoch 006 - training loss (CrossEntropy): 1862.3724, validation CrossEntropy: 0.6391
2025-05-21 21:44:17 [INFO]: Epoch 007 - training loss (CrossEntropy): 1855.3852, validation CrossEntropy: 0.6266
2025-05-21 21:44:36 [INFO]: Epoch 008 - training loss (CrossEntropy): 1848.6624, validation CrossEntropy: 0.6175
2025-05-21 21:44:54 [INFO]: Epoch 009 - training loss (CrossEntropy): 1842.3058, validation Cros

Testing classification metrics: 
ROC_AUC: 0.9007518796992481, 
PR_AUC: 0.9232825513565572,
F1: 0.9023668639053254,
Precision: 0.8879184861717613,
Recall: 0.9172932330827067,
Accuracy: 0.9007518796992481


## Not deep models

In [72]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.base import ClassifierMixin
from typing import Any, TypeVar
from collections import namedtuple
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

In [86]:
_T = TypeVar('_T', bound=ClassifierMixin)

def evaluate_model(model: ClassifierMixin, X: Any, y: Any):
    y_pred = model.predict(X)
    accuracy = accuracy_score(y, y_pred)
    f1 = f1_score(y, y_pred, average='macro')
    precision = precision_score(y, y_pred, average='macro')
    recall = recall_score(y, y_pred, average='macro')
    confusion = confusion_matrix(y, y_pred)
    return namedtuple('Evaluation', ['accuracy', 'f1', 'precision', 'recall', 'confusion'])(accuracy, f1, precision, recall, confusion)

def train_model(
        model_cls: _T, 
        model_kwargs: dict[str, Any],
        X_train: np.ndarray,
        y_train: np.ndarray,
        X_test: np.ndarray,
        y_test: np.ndarray
    ) -> _T:
    model = model_cls(**model_kwargs)
    if X_train.ndim == 3:
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1] * X_train.shape[2]))
    if X_test.ndim == 3:
        X_test = X_test.reshape((X_test.shape[0], X_test.shape[1] * X_test.shape[2]))

    X_train = np.nan_to_num(X_train, nan=-1.0)
    X_test = np.nan_to_num(X_test, nan=-1.0)
    model.fit(X_train, y_train)
    train_metrics = evaluate_model(model, X_train, y_train)
    val_metrics = evaluate_model(model, X_test, y_test)
    print(f"Model - {model_cls.__name__}")
    print("\tTrain metrics:")
    print(f"\t\tAccuracy: {train_metrics.accuracy:.4f}")
    print(f"\t\tF1: {train_metrics.f1:.4f}")
    print(f"\t\tPrecision: {train_metrics.precision:.4f}")
    print(f"\t\tRecall: {train_metrics.recall:.4f}")
    print("\tValidation metrics:")
    print(f"\t\tAccuracy: {val_metrics.accuracy:.4f}")
    print(f"\t\tF1: {val_metrics.f1:.4f}")
    print(f"\t\tPrecision: {val_metrics.precision:.4f}")
    print(f"\t\tRecall: {val_metrics.recall:.4f}")
    return model

In [87]:
svc = train_model(
    SVC,
    {},
    X_train=train_data["X"],
    y_train=train_data["y"],
    X_test=test_data["X"],
    y_test=test_data["y"],
)

Model - SVC
	Train metrics:
		Accuracy: 0.6802
		F1: 0.6641
		Precision: 0.7228
		Recall: 0.6802
	Validation metrics:
		Accuracy: 0.6857
		F1: 0.6606
		Precision: 0.7639
		Recall: 0.6857


In [88]:
rfc = train_model(
    RandomForestClassifier,
    {},
    X_train=train_data["X"],
    y_train=train_data["y"],
    X_test=test_data["X"],
    y_test=test_data["y"],
)

Model - RandomForestClassifier
	Train metrics:
		Accuracy: 1.0000
		F1: 1.0000
		Precision: 1.0000
		Recall: 1.0000
	Validation metrics:
		Accuracy: 0.9459
		F1: 0.9459
		Precision: 0.9461
		Recall: 0.9459


In [89]:
xgb = train_model(
    XGBClassifier,
    {},
    X_train=train_data["X"],
    y_train=train_data["y"],
    X_test=test_data["X"],
    y_test=test_data["y"],
)

Model - XGBClassifier
	Train metrics:
		Accuracy: 1.0000
		F1: 1.0000
		Precision: 1.0000
		Recall: 1.0000
	Validation metrics:
		Accuracy: 0.9308
		F1: 0.9307
		Precision: 0.9335
		Recall: 0.9308
